# Настройка нейросетей

### Функции потерь
* Мы использовали функцию потерь $L(y, \tilde y) = (y - \tilde y)^2$ при обучении перцептронов.
* При разборе устройства классификатора использовали
\begin{equation*}
    L(k,l)  = \left\lbrace 
        \begin{array}{rl}
            0, & \mbox{если } l= k \mbox{ (корректный ответ)}\\
            1, & \mbox{если } l\neq k \mbox{ и } l\in \{1,2,\dots,K\} \\
            d, & \mbox{если } l=\mathscr{D} \mbox{ (в затруднении)}.
        \end{array}  \right. 
\end{equation*}
* В коде примеров на TensorFlow использовали перекрестную энтропию $L(p,q)=-\Sigma_x{p(x) \log q(x)}$.
* ...

Все эти функции показывают, насколько плоха наша модель, и мы обучали модель, минимизируя фунции потерь.

**Выбор функции потерь - важный аспект проектирования нейросети.**

### Метод максимального правдоподобия для НС

Нейросеть можно рассматривать как вероятностную модель. Она принимает на вход параметры объекта $x$, возвращает ответ $y$, который можно рассматривать как случайную величину.

Т.е. нейросетевая модель строит распределение вероятностей
$$
p(y | x; \theta),
$$
$\theta$ -- независимые параметры модели.

Как и с обычной вероятностной моделью мы можем воспользоваться методом максимального правдоподобия для оценки параметров.

#### Метод максимального правдоподобия

Есть входные примеры $x_1, x_2, \dots, x_n$ и ответы $y_1, y_2, \dots, y_n$. Рассчитаем вероятность получения ответов $y_1, y_2, \dots, y_n$ при известных $x_1, x_2, \dots, x_n$:

$$
P(\theta) = p(y_1 | x_1; \theta) p(y_2 | x_2; \theta) \dots p(y_n | x_n; \theta)
$$
нужно подобрать параметры $\theta$, чтобы получить максимальную вероятность $P(\theta)$.

Обычно имеют дело с отрицательным (у нас задача минимизации) логарифмом вероятностей $J(\theta) = -\log P(\theta)$:

$$
J(\theta) = - \sum_{i=1}^n \log p(y_i | x_i; \theta)
$$

*Например, для регрессии $p_{model}(y | x) = N(y, f(x, \theta), I)$ данный метод приводит к функции потерь $L=(y -f(x, \theta))^2$*

Зачем такой подход нужен?

* Единообразие: нет необходимости проектировать функцию потерь заново под каждую задачу.
* Есть функции активации с насыщением, в которых фигурирует $e^u$ => изчезающий градиент. Если использовать логарифмическую функцию потерь, логарифм компенсирует экспоненту, градиент везде "хороший".


## Функции активации

* Выходные блоки.
* Скрытые блоки.

### Выходные блоки

Выбор функции активации выходного блока и функции потерь тесно связаны.

Пусть $h$ -- выходы скрытого слоя, $W$ -- весовые коэффициенты, $b$ -- пороги.

#### Задача регрессии 

Линейная функция активации, порождающая условное среднее:
$$
p(y|x) = N(y, W h + b, I).
$$
Минимизация максимального правдоподобия эквивалентна минимизации среднеквадратической ошибки
$$
\frac{1}{n}\sum_{i-1}^n(y_i - \hat y_i)^2
$$


#### Прогноз бинарной величины. 

Рассматриваем выход сети как вероятность $\hat y = P(y=1| x)$ => функция активации сигмоидальная: $sigm(Wh + b)$


**Не надо** использовать функцию потерь 
$$(y- sigm(Wh + b))^2.$$

Метод логарифмического правдоподобия приведет к формуле:
$$
J(\theta) = \log[1 + e^{(1-2y)(Wh + b)} ].
$$
В этом случае насыщение достигается, если ответ уже получен

#### Номинальные величины

* Ранее была бинарная величина:  $\hat y = P(y=1| x)$? и сигмоидная функция активации.
* Сейчас несколько: $\hat y_i = P(y=i| x)$. При этом нужно:
 1. каждый $\hat y_i \in [0, 1]$
 2. $\sum \hat y_i = 1$

Воспользуемся функцией $softmax$:

$$
softmax(z_i) = \frac{e^{z_i}}{\sum_i e^{z_i}} 
$$

Метод логарифмического правдоподобия будет удобен, т.к. логарифм "уничтожит" экспоненту:
$$
log(softmax(z_i)) = z_i - log(\sum e^{z_i}).
$$

Насыщения нет.

### Общие рекомендации по функциям активации

1. Сигмоида:
 (+) ограничена;
 (-) насыщение;
 (-) не центрирована в 0;
 (-) $exp$ дорогая.

2. Гиперболический тангенс:
 (+) ограничена;
 (-) насыщение;
 (+) центрирована;
 (-) $exp$ дорогая.
 
3. ReLU:
 (+) нет насыщения;
 (+) вычислительно проста;
 (+) быстро сходится;
 (-) не центрирована в 0;
 (-) нет производной в 0;
 (-) "мертвые" нейроны.
 
4. LeakyReLU:
 $\max(0.001 x, x)$

## Регуляризация

Регуляризация -- стратегия уменьшения ошибки на этапе обобщения за счет увеличения ошибки на этапе обучения.

* Обучение усложняется, когда размерность данных велика.

* Чтобы алгоритм обучения хорошо обобщался (прогноз на тестовых данных), нужно иметь априорные знания о тех функциях, которые он должен изучить.

* Частое предположение: гладкость функций, локальное постоянство. (Функция не должна сильно изменяться в на небольшом участке).


### Штрафы по нормам параметров

Регуляризированная функция потерь складывается из целевой функции и штрафа на величины (обычно нормы) параметров:
$$
\tilde J(\theta, X, y) = J(\theta, X, y) + \alpha \Omega(\theta),
$$
где $\alpha \in [0, 1)$ -- параметр "силы" регуляризации.

#### Какие параметры регулиризуем?

 * Обычно штрафуют только веса $W$.
 * Пороги оставляют "как есть":
   * для подбора порогов обычно нужно меньше примеров, чем для весов (вес -- взаимодействие двух переменных, порог - одной);
   * ограничение порогов может быть причиной сильного недообучения.
 

### Регуляризация по норме $L^2$

$$
\tilde J(\theta, X, y) = J(\theta, X, y) + \alpha w^T w
$$

![Регуляризация](img/tuning/regularization.png)

### Регуляризация по норме $L^1$

$$
\tilde J(\theta, X, y) = J(\theta, X, y) + \alpha ||w||
$$



## Ансамбли моделей

* Уменьшаем ошибку обобщения за счет комбинации нескольких моделей: 
  * обучаем раздельно несколько моделей, 
  * устраиваем голосование между ними.
  
(Разные модели ошибаются по-разному)

Например, $k$ моделей регрессии, каждая модель делает ошибку $\epsilon_i$ на $i$-м примере. Пусть для простоты ошибки нормально распределены с нулевым средним, дисперсиями $E(\epsilon_i) = v$ и ковариациями $E(\epsilon_i, \epsilon_j) = c$.

 * Ошибка предсказания ансамбля: $\frac 1k \sum_i \epsilon_i$.
 * Матожидание квадрата ошибки: $\frac 1k v + \frac{k-1}{k}c$.

Два крайних случая:
1. При идеальной корреляции ($c=v$), среднеквадратическая ошибка равна $v$ (усреднение не помогает). 
2. При некоррелированных ошибках ($c=0$) среднеквадратическая ошибка равна $\frac 1k v$.


## Прореживание (Dropout)

Эмпирический метод создания ансамблей моделей на базе нейросетей.
![Прореживание](img/tuning/dropout.png)

1. Каждый параметр сети при обучении может быть включен или выключен. 
2. При загрузке примера для обучения мы формируем случайную битовую маску, маркирующую, будет ли использоваться соотвествующий параметр в сети при обработке примера. Вероятность $p_{param=1}$ -- гиперпараметр алгоритма, обычно 0.5.
3. Подаем пример, производим прямой и обратный проходы обучения (с учетом выключенных связей). 

Обученную таким образом сеть можно рассматривать, как ансамбль из огромного числа подсетей, частично разделяющих между собой параметры. Такая сеть более устойчива к шумам, менее подвержена переобучению и т.п.

После обучения сети ее нужно "собрать обратно", включив все связи. Работает эмпирическое правило: если при обучении вероятность включения параметров была равна $p$, то вход в нейрон "собранной" сети будет выше в 1/p раз, чем при обучении. Поэтому в итоговой сети веса делят на $p$.

## Сеть не обучается?

1. Начинайте с самой простой модели, про которую известно, что она работает с вашими данными. Используйте стандартные функции потерь.    
2. Сначала отключите все дополнительные "фишки" (регуляризацию, сгенерированные (аугментированные) данные, прореживание и т.п.).
3. Убедитесь, что входные данные корректны.
4. Обучитесь на маленьком наборе данных (2-20 примеров). Убедитесь, что сеть может воспроизвести их.
5. Шаг за шагом добавляйте обратно все отключенные части (регуляризацию, свою функцию потерь и т.д.).

[Сборник советов по диагностике работы нейросетей](https://blog.slavv.com/37-reasons-why-your-neural-network-is-not-working-4020854bd607)

## Порядок важности настроек

1. Архитектура сети (включая функции активации, функцию потерь, метод обучения).
2. Параметры, отвечающие за скорость обучения.
3. Регуляризация.